In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from os import path

root_path = "/content/gdrive/Shareddrives/BA/R Python/Đồ án/src/"
dataset_path = path.join(root_path, "dataset")
annotated_path = path.join(dataset_path, 'val_annotated')
model_path = path.join(root_path, "models")

!ls "$dataset_path"
!ls "$annotated_path"
!ls "$model_path"

annotated  raw	val_annotated
inout  star


In [ ]:
from transformers import pipeline

In [ ]:
inout_cls = pipeline(task="text-classification", model = path.join(model_path, "inout"), device = "cpu")
star_cls = pipeline(task="text-classification", model = path.join(model_path, "star"), device = "cpu")

In [ ]:
!pip install underthesea py_vncorenlp ftfy emot

In [ ]:
import emot
emot_obj = emot.core.emot()

In [ ]:
import py_vncorenlp

py_vncorenlp.download_model(save_dir='/content')
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/content')

In [ ]:
!git clone https://github.com/v-nhandt21/Vinorm.git
!ls

In [ ]:
import os

slangs_dict = {}
for file in os.listdir('Vinorm/vinorm/Mapping'):
  if file not in ['Teencode.txt', 'Number.txt']:
    continue

  print(file, ':', end = ' ')
  file_path = 'Vinorm/vinorm/Mapping/' + file

  with open(file_path, 'r', encoding = 'utf8') as f:
    data_slangs = f.readlines()
    print(len(data_slangs))

    for i in data_slangs:
      i = i.replace("\n", "").lower().strip()
      i = i.split("#")

      for punc in ['(', ')', '.', '+', '*']:
        i[0] = i[0].replace(punc, '\\' + punc)

      slangs_dict[i[0]] = i[1]

slangs_dict.update({
    'sp': 'sản phẩm',
    'tl': 'trả lời',
    'mn': 'mọi người',
    'mng': 'mọi người',
    'mngf': 'mọi người',
    'h': 'giờ',
    'v': 'vậy',
    'cgi': 'cái gì',
    'vs': 'với',
    'sv': 'sinh viên',
    'sop': 'shop',
    'lmj': 'làm gì',
    'e': 'em',
    'dc': 'được',
    'nx': 'nữa',
    'kg': 'không',
    'bt': 'biết',
    'l': 'làm',
    'ntin': 'nhắn tin',
    'j': 'gì',
    'v': 'vậy',
    'lm': 'làm',
    't': 'tao',
    's': 'sao',
    'ngta': 'người ta',
    'z': 'vậy',
    'nt': 'nhắn tin',
    'ib': 'inbox',
    'lh': 'liên hệ',
    'kh': 'không'
})

len(slangs_dict)

In [ ]:
VIETNAMESE_TEXT = 'àáãạảăắằẳẵặâấầẩẫậèéẹẻẽêềếểễệđìíĩỉịòóõọỏôốồổỗộơớờởỡợùúũụủưứừửữựỳỵỷỹýÀÁÃẠẢĂẮẰẲẴẶÂẤẦẨẪẬÈÉẸẺẼÊỀẾỂỄỆĐÌÍĨỈỊÒÓÕỌỎÔỐỒỔỖỘƠỚỜỞỠỢÙÚŨỤỦƯỨỪỬỮỰỲỴỶỸ'
LATIN_VI = r'[A-Za-z' + VIETNAMESE_TEXT + r']'

regex_dict = {
    r'(?<=\d+\s)k\b': 'nghìn',
    r'(?<=\d+\s)m(?=\s\d+)\b': 'mét',
    r'(?<=\d+\s)kg\b': 'kilogram',
    r'(?<=size\s)s\b': 'nhỏ',
    r'(?<=size\s)m\b': 'trung bình',
    r'(?<=size\s)l\b': 'lớn',
    r'(?<=size\s)xl\b': 'rất lớn',
    r'(?<=size\s)xxl\b': 'rất rất lớn',
}

In [ ]:
import underthesea as u
import ftfy
import string
from vinorm import TTSnorm
import unicodedata

def demoji(text):

  text = text.strip()
  text = ' ' + text + ' '

  emojis = emot_obj.emoji(text)
  emoticons = emot_obj.emoticons(text)
  for k, v in zip(emojis['value'] + emoticons['value'], emojis['mean'] + emoticons['mean']):

    for punc in ['(', ')', '.', '+', '*']:
      k = k.replace(punc, '\\' + punc)

    v = v.replace(':', '')
    v = '<emoji>' + v + '</emoji>'

    pattern = r'\s' + k + r'\s'
    text = re.sub(pattern, ' ' + v + ' ', text)

  text = text.lower()
  text = text.strip()

  return text

def pre_processing(text):

  text = text.lower()
  text = unicodedata.normalize('NFC', text)

  # Clean space
  text = text.replace("\n", " ")
  text = re.sub(r'\s+', ' ', text)

  # Fix encoding, punc
  text = text.strip()
  text = ftfy.fix_encoding(text)
  text = ftfy.fix_text(text)
  text = u.text_normalize(text)

  # Turn emoji, emoticon to text
  text = demoji(text)

  # Fix dup punc
  text = re.sub(r'([!"#$%&\'()*+,\-\.\/:;<=>?@[\\\]^_`{|}~])(?:\s*\1)+', r' \1 ', text)
  text = re.sub(r'\s+', ' ', text)

  # Remove punc
  text = re.sub(r'[!"#$%&\'()*+,\-\.:;=?@[\\\]^_`{|}~]', ' ', text)
  text = re.sub(r'<(?!\/?[^\s]+>)|(?<!<\/?[^\s]+)>|(?<!<)\/(?![^\s]+>)', ' ', text)
  text = re.sub(r'\s+', ' ', text)

  # Normalize with regex
  text = re.sub(r'((?<!' + LATIN_VI + r'\s?)' + LATIN_VI + r'{1,3}(?!' + LATIN_VI + r'\s?))', r' \1 ', text)
  text = text.strip()
  text = re.sub(r'\s+', ' ', text)
  for pattern, value in regex_dict.items():
    text = re.sub(pattern, value, text)

  # Fix slang
  text = " ".join([slangs_dict.get(word, word) for word in text.split()])

  # Word segment
  try:
    text = rdrsegmenter.word_segment(text)[0]
  except:
    pass

  # Fix token
  text = re.sub(r'<\s?(\/)?\s([^\s]+)\s>', r'<\1\2>', text)

  text = re.sub(r'\s+', ' ', text)
  return text.strip()


In [ ]:
while True:
  text = input("INPUT: ")

  if text in ['', 'e']:
    break

  text = pre_processing(text)

  inout = inout_cls(text)[0]['label']

  if inout == 'out':
    print("OUT DOMAIN")
    print()
    continue

  star = star_cls(text)[0]['label']
  print("STAR:", star)
  print()

OUT DOMAIN

STAR: 2

STAR: 4

OUT DOMAIN

STAR: 1

